In [ ]:
from music21 import *
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor, LazyClassifier

Reading file

In [ ]:
file = converter.parse('JeSuisRobert.mid')

Retrieving instruments

In [ ]:
instruments =  instrument.partitionByInstrument(file) 

Retrieving notes

In [ ]:
notes = []
for element in file.recurse():
    if isinstance(element, note.Note):
        notes.append(element)

Converting notes to numbers

In [ ]:
single_values = []
for itm in notes:
    pt = str(itm.pitch)
    ptc = str(itm.pitch.pitchClass)
    pto = str(itm.pitch.octave)
    dro = str(itm.duration.ordinal)
    drt = str(itm.duration.type)
    abs_pit = (itm.pitch.octave * 12) + itm.pitch.pitchClass
    abs_dro = (int(dro) * 512)
    abs_numb = abs_dro + abs_pit
    single_values.append(abs_numb)
    

Creating dictionaries from notes (from 1 to 4 notes)

In [ ]:
two_notes = {}
three_notes = {}
four_notes = {}
all_values = len(single_values)
for index in range(all_values):
        if (index < all_values - 3):
            two_notes[len(two_notes)] = [[(single_values[index] ^ 2) + single_values[index + 1]], [single_values[index + 2]]]
            if (index < all_values - 4):
                three_notes[len(three_notes)] = [[(single_values[index] ^ 3) + (single_values[index + 1] ^ 2) + single_values[index + 2]], [single_values[index + 3]]]
                if (index < all_values - 5):
                    four_notes[len(four_notes)] = [[(single_values[index] ^ 4) + (single_values[index + 1] ^ 3) + (single_values[index + 2] ^ 2)  + single_values[index + 3]], [single_values[index + 4]]]


Creating data frames from dictionaries

In [ ]:
df_two_notes = pd.DataFrame.from_dict(two_notes, orient='index')
df_three_notes = pd.DataFrame.from_dict(three_notes, orient='index')
df_four_notes = pd.DataFrame.from_dict(four_notes, orient='index')

df_two_notes.head()
#df_three_notes.head()
#df_four_notes.head()


Creating train test split

In [ ]:
y_two_notes = df_two_notes.iloc[:,1]
X_two_notes = df_two_notes.iloc[:,0]

#print(X_two_notes)
#X_two_notes.head()


Call train test split on the data and capture the results

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_two_notes, y_two_notes, random_state=42, test_size=0.2)

LazyClassifier Instance and fiting data

In [ ]:
cls= LazyClassifier(ignore_warnings=False, custom_metric=None)
models, predictions = cls.fit(X_train, X_test, y_train, y_test)